In [ ]:
import EEGReptile as re   # import Reptile-EEG lib
import copy

Prepare data and saving folders for experiment

In [ ]:
experiment_name = re.experiment_storage(experiment_name='BCI_IV_EEGNet_example')   #create storage for models and new experiment data
dataset = re.MetaDataset(dataset_name='BCI_IV')   #load dataset created in load data example
print('Dataset ' + dataset.dataset_name + ' loaded.' + ' subjects: ' + str(dataset.subjects))
targ = [7] # choose target subject to perform tests (Zero-. Few-shot) later
tr_sub = copy.deepcopy(dataset.subjects)
for sub in targ:   # remove target subject from 
    tr_sub.remove(sub) 

Prepare the model

In [ ]:
mparams = {   # params of EEGNet model
    "model_type": "inEEG_net",
    "num_classes": 4,
    "dropout1": 0.52,
    "dropout2": 0.36,
    "f1": 16,
    "sampling_rate": 250,
    "num_channels": 22,
    "depth_multiplier": 12,
    "time_of_interest": 500,
    "time_points": 625,
    "lowpass": 50,
    "point_reducer": 5
}
model = re.model_from_params(mparams) # crate codel with 

Find the meta-learning hyperparameters

In [ ]:
params = re.meta_params(metadataset=dataset, model=model, tr_sub=tr_sub, trials=100,
                        experiment_name=experiment_name)

Perform meta-training (pretrained models will be saved in experiment folder)

In [ ]:
pretraining_auc = re.meta_exp(params=params, model=model, target_sub=targ + [3], # here we add subj 3 to use him in parameters search for fine-tuning 
                              metadataset=dataset, mode='batch', meta_optimizer=False, num_workers=2,
                              experiment_name=experiment_name,
                              baseline= True)  # train or not similar network on same data with classic Transfer Learning
print('Pretraining completed. Mean auc for pretraining: ' + str(pretraining_auc))

Find hyperparams for fine-tuning
-- lr
-- (ax + b) = Epochs, where x is number of datapoints for fine-tuning

In [ ]:
af_params = re.aftrain_params(metadataset=dataset, model=model, tst_subj=[3], trials=100, jobs=2,
                              experiment_name=experiment_name)

Fine-tuning experiment (evaluation of performance on unseen subj)
(results will be stored in experiment folder)

In [ ]:
re.aftrain(target_sub=targ, model=model,
           af_params=af_params, metadataset=dataset,
           length=65, # max number of datapoints which will be used for fine-tuning
           iterations=5, # number of similar experiments with different random seed to get the mean stats
           experiment_name=experiment_name, last_layer=False)